## A Basic Introduction to  Julia

This Introduction is based on [Chris Rackauckas'](www.stochasticlifestyle.com) excellent [IntroToJulia]([http://ucidatascienceinitiative.github.io/IntroToJulia/) from which I freely copy and paste here.

This quick introduction assumes that you have basic knowledge of some scripting language and provides an example of the Julia syntax. So before we explain anything, let's just treat it like a scripting language, take a head-first dive into Julia, and see what happens.

You'll notice that, given the right syntax, almost everything will "just work". There will be some peculiarities, and these we will be the facts which we will study in much more depth. Usually, these oddies/differences from other scripting languages are "the source of Julia's power".

### This Notebook

* The code in this notebook is live, i.e. it works.
* You can execute a cell by clicking inside it, and then clicking on the "play" button, or you just hit Ctrl+Enter.
* I encourage you to execute and play around with all code cells!

### Problems

Time to start using your noggin. Scattered in this document are problems for you to solve using Julia. Many of the details for solving these problems have been covered, some have not. You may need to use some external resources:

https://docs.julialang.org/

https://gitter.im/JuliaLang/julia

Your main forum is [https://discourse.julialang.org](https://discourse.julialang.org)

There is a nice [Slack Channel](https://julialang.slack.com)

Solve as many or as few problems as you can. Please work at your own pace, or with others if that's how you're comfortable!

## Documentation and "Hunting"

The main source of information is the [Julia Documentation](http://docs.julialang.org/en/latest/manual/). Julia also provides lots of built-in documentation and ways to find out what's going on. The number of tools for "hunting down what's going on / available" is too numerous to explain in full detail here, so instead this will just touch on what's important. For example, the `?` gets you to the documentation for a type, function, etc.

In [ ]:
?copy


to see what *methods* are defined by a certain name, use `methods`. Let's see how the methods for the function `+`

In [ ]:
methods(+)

In [ ]:
fieldnames(LinSpace)

and find out which method was used with the `@which` macro:

In [ ]:
@which copy([1,2,3])

copy<i>{T<:Array{T,N}}</i>(a::<b>T</b>) at <a href="https://github.com/JuliaLang/julia/tree/9c76c3e89a8c384f324c2e0b84ad28ceef9ab69d/base/array.jl#L70" target="_blank">array.jl:70</a>



Notice that this gives you a link to the source code where the function is defined.

Lastly, we can find out what type a variable is with the `typeof` function:

In [ ]:
a = [1.0;2;3]
typeof(a)

We can also get a direct view of source code with

In [ ]:
@edit copy([1,2,3])

### Array Syntax

The array syntax is similar to MATLAB's conventions.

In [ ]:
a = Vector{Float64}(5) # Create a length 5 Vector (dimension 1 array) of Float64's

a = [1;2;3;4;5] # Create the column vector [1 2 3 4 5]

a = [1 2 3 4] # Create the row vector [1 2 3 4]

a[3] = 2 # Change the third element of a (using linear indexing) to 2

b = Matrix{Float64}(4,2) # Define a Matrix of Float64's of size (4,2)

c = Array{Float64}(4,5,6,7) # Define a (4,5,6,7) array of Float64's 

e = falses(4,4)

f = trues(5,2)

#checkout more at http://docs.julialang.org/en/stable/manual/arrays/

mat    = [1 2 3 4
          3 4 5 6
          4 4 4 6
          3 3 3 3] #Define the matrix inline 

mat[1,2] = 4 # Set element (1,2) (row 1, column 2) to 4

mat


Note that, in the console (called the REPL), you can use `;` to surpress the output. In a script this is done automatically. Note that the "value" of an array is its pointer to the memory location. **This means that arrays which are set equal affect the same values**:

In [ ]:
a = [1;3;4]
b = a
b[1] = 10
a

To set an array equal to the values to another array, use copy

In [ ]:
a = [1;4;5]
b = copy(a)
b[1] = 10
a

We can also make an array of a similar size and shape via the function `similar`, or make an array of zeros/ones with `zeros` or `ones` respectively:

In [ ]:
c = similar(a)
d = zeros(a)
e = ones(a)
f = trues(3)
println(c); println(d); println(e); println(f)

In [ ]:
rand(2,2)
randn(2,2)

In [ ]:
fill!(a,10)

In [ ]:
# concatenation: how to combine arrays?
a = ones(2)
b = rand(3)
cat(1,a,b)  # same as vcat(a,b)

In [ ]:
a = reshape(ones(8),2,2,2)
b = reshape(zeros(8),2,2,2)
z = cat(3,a,b)  # concat along the third dim
z[:,:,1]
z[:,2,:]

#### Array Indexing

You can index arrays in 2 ways: linear index, and multidimensional index. In general, the linear index is faster.

In [35]:
z = rand(3,3,3)
z[1,3,2]  # multidimensional
z[4]  # linear index: julia stores columnwise
using Base.Test
@test z[1,3,2] == z[sub2ind(size(z),1,3,2)]

Test Passed


Arrays can be of any type, specified by the type parameter. One interesting thing is that this means that arrays can be of arrays:

In [ ]:
a = Vector{Vector{Float64}}(3)
a[1] = [1;2;3]
a[2] = [1;2]
a[3] = [3;4;5]
a

You can create empty arrays and push onto them:

In [ ]:
x = Int[]
for i in 10:-2:0
    println("x is now equal to $x")
    push!(x,i)   # remember that ! for later.
end
x

y = zeros(Int,6)
vals = 10:-2:0
for i in 1:6
    y[i] = vals[i]
    println("y is = $y")
end

### Array Comprehensions

A very powerful way to generate arrays in julia are [comprehensions](http://docs.julialang.org/en/stable/manual/arrays/#comprehensions). Imagine you want to fill a (3,5,6) array with the values of a function `foo` that takes 3 arguments (one for each dimension):

In [ ]:
foo(x,y,z) = sin(x)*0.5y^0.5 + z^2
d = [foo(i,j,k) for i in 1:3, j in linspace(0.01,0.1,5), k in [log(l) for l in 2:7]];

In [ ]:
# generator expressions work in a similar way
# just leave the square brackets away
sum(1/n^2 for n=1:1000)  # this allocates no temp array

In [ ]:
# can have indices depend on each other
[(i,j) for i=1:3 for j=1:i]   # this is always 1D

In [ ]:
# you can even condition on the indices
[(i,j) for i=1:3 for j=1:i if i+j == 4]

### iterators

* eachindex
* indices
* linearindices

### sparse

In [ ]:
### views and subarrays

### Broadcasting

You can perform operations element-wise on an array with the `.` syntax. For example, do `A .+ B` to sum elements of A and B elementwise. Of course they need to be compatible dimensionwise.

Julia v0.6 is now able to broadcast any function over an array in an efficient manner:

In [ ]:
fff(x) = x^(2/3)
@test fff.(d) == broadcast(fff,d)

---------------------

#### Question 1

Can you explain the following behavior? Julia's community values consistency of the rules, so all of the behavior is deducible from simple rules. (Hint: I have noted all of the rules involved here).

In [ ]:
a

In [ ]:
b = a
println("a=$a")
b[1] = [1;4;5]
a

----------------------------------------

To fix this, there is a recursive copy function: `deepcopy`

In [ ]:
b = deepcopy(a)
b[1] = [1;2;3]
a

For high performance, Julia provides mutating functions. These functions change the input values that are passed in, instead of returning a new value. By convention, mutating functions tend to be defined with a `!` at the end and tend to mutate their first argument. An example of a mutating function in `scale!` which scales an array by a scalar (or array)

In [ ]:
a = [1;6;8]
scale!(a,2) # a changes

The purpose of mutating functions is that they allow one to **reduce the number of memory allocations** which is crucial for achiving high performance.

## Control Flow

Control flow in Julia is pretty standard. You have your basic for and while loops, and your if statements. There's more in the documentation.

In [ ]:
for i=1:5 #for i goes from 1 to 5
    println(i)
end

t = 0
while t<5
    println(t)
    t+=1 # t = t + 1
end

school = :ScPo

if school==:ScPo & (x == 0)
    println("your mascott is The Lion and the Fox")
else
    println("you are not from ScPo, I don't know what your mascott is")
end

One interesting feature about Julia control flow is that we can write multiple loops in one line:

In [ ]:
for i=1:2,j=2:4
    println(i*j)
end

## Problems

You can try to solve those problems either in a julia REPL (your terminal), in Juno, or by adding new cells in this notebook.

#### Problem 1a

Use Julia's array and control flow syntax in order to define the NxN Strang matrix:

$$ \left[\begin{array}{ccccc}
-2 & 1\\
1 & -2 & 1\\
 & \ddots & \ddots & \ddots\\
 &  & \ddots & \ddots & 1\\
 &  &  & 1 & -2
\end{array}\right] $$

#### Problem 1b

Write an expression that computes

$$ y = \sum_{i=1}^{25} \sum_{j=1}^i j^{-1}  $$


#### Problem 2

```julia
#### Prepare Data

X = rand(1000, 3)               # feature matrix
a0 = rand(3)                    # ground truths
y = X * a0 + 0.1 * randn(1000);  # generate response
```

Given an Nx3 array of data (`randn(N,3)`) and a Nx1 array of outcomes, produce the data matrix `X` which appends a column of 1's to the data matrix, and solve for the 4x1 array `β` via `βX = b` using `qrfact` or `\`. (Note: This is linear regression)

#### Problem 3

Compare your results to that of using `llsq` from `MultivariateStats.jl` (note: you need to go find the documentation to find out how to use this!)


#### Problem 4

The logistic difference equation is defined by the recursion

$$ b_{n+1}=r*b_{n}(1-b_{n}) $$

where $b_{n}$ is the number of bunnies at time $n$. Starting with $b_{0}=.25$, by around $400$ iterations this will reach a steady state. This steady state (or steady periodic state) is dependent on $r$. Write a function which solves for the steady state(s) for each given $r$, and plot "every state" in the steady attractor for each $r$ (x-axis is $r$, $y$=value seen in the attractor) using Plots.jl. Take $r\in\left(2.9,4\right)$




## Functions

In [ ]:
f(x,y) = 2x+y # Create an inline function

In [2]:
f(1,2) # Call the function

4

In [3]:
function f(x)
  x+2  
end # Long form definition

f (generic function with 2 methods)

By default, Julia functions return the last value computed within them.

In [4]:
f(2)

4

### Argument passing

* julia does not copy the value of a function argument
* it assigns a new binding
* notice that this does not mean that it will assign values in the calling frame:

In [32]:
f_change(x) = x = 5
y = 1
f_change(y)
y

1

In [33]:
# we have already seen this is different with arrays.
f_change!(x::Vector) = x[1] = 0
y = rand(2)
f_change!(y)
y

2-element Array{Float64,1}:
 0.0     
 0.228279

### Operators are Functions

* *Operators* are things like `+,-,*,/` but also `[A B], 1:n` etc 

In [36]:
@test 1+2+3 == +(1,2,3)

Test Passed


###Anonymous functions

* an anonymous function is a function without a name
* They are useful as input to *other* functions
* we will encounter that when using optimizers, this is useful to pass constant parameters to an optimization algorithm
* an anynomous function creates a *closure*.
* notice that the compiler assigns a consecutive number to each new anonymous function you create:

In [37]:
# use the arrow syntax
x -> x^2 + 3*pi

(::#1) (generic function with 1 method)

In [40]:
# can have multiple args
(x,y) -> 2x - 3y
# use it with another function
map(x -> x^2 + 3*pi, [1,2,3])

3-element Array{Float64,1}:
 10.4248
 13.4248
 18.4248

### Tuples and named Tuples

* These are pairs/triples/... of arbitrary values
* you can access the values with square brackets (like an array) but you cannot change the values (unlike an array)
* function args are tuples

In [43]:
tup = (1,pi,:hi)
typeof(tup)

Tuple{Int64,Irrational{:π},Symbol}

In [49]:
# named tuples: arriving in version 0.7 (and 1.0!)
if VERSION > v"0.7.0"
#     ntup = (one=1,pi=pi,say_hi=:hi)
    #ntup
else
    println("named tuples haven't arrived yet, you are on version $VERSION")
end

named tuples haven't arrived yet, you are on version 0.6.0


### when is version 1.0 coming?

* not missing much! [https://github.com/JuliaLang/julia/milestone/4](https://github.com/JuliaLang/julia/milestone/4)

### Multiple Dispatch

A key feature of Julia is **multiple dispatch**. Notice here that there is "one function", `f`, with two methods. Methods are the actionable parts of a function. Here, there is one method defined as `(::Any,::Any)` and `(::Any)`, meaning that if you give `f` two values then it will call the first method, and if you give it one value then it will call the second method.

Multiple dispatch works on types. To define a dispatch on a type, use the `::Type` signifier:

In [5]:
f(x::Int,y::Int) = 3x+2y

f (generic function with 3 methods)

Julia will dispatch onto the strictest acceptible type signature.

In [6]:
f(2,3) # 3x+2y

12

In [7]:
f(2.0,3) # 2x+y since 2.0 is not an Int

7.0

Types in signatures can be parametric. For example, we can define a method for "two values are passed in, both Numbers and having the same type". Note that `<:` means "a subtype of".

In [8]:
f(x::T,y::T) where T = 4x+10y 

f (generic function with 4 methods)

In [9]:
f(2,3) # 3x+2y since (::Int,::Int) is stricter

12

In [10]:
f(2.0,3.0) # 4x+10y

38.0

Note that type parameterizations can have as many types as possible, and do not need to declare a supertype. For example, we can say that there is an `x` which must be a Number, while `y` and `z` must match types:

In [15]:
f(x::T,y::T2,z::T2) where {T<:Number,T2} = 5x + 5y + 5z

f (generic function with 6 methods)

We will go into more depth on multiple dispatch later since this is the core design feature of Julia. The key feature is that Julia functions specialize on the types of their arguments. This means that `f` is a separately compiled function for each method (and for parametric types, each possible method). The first time it is called it will compile.

-------------------------

#### Question 2

Can you explain these timings?

In [ ]:
f(x,y,z,w) = x+y+z+w
@time f(1,1,1,1)
@time f(1,1,1,1)
@time f(1,1,1,1)
@time f(1,1,1,1.0)
@time f(1,1,1,1.0)

4.0




-------------------------

Note that functions can also feature optional arguments:

In [ ]:
function test_function(x,y;z=0) #z is a keyword argument
  if z==0
    return x+y,x*y #Return a tuple
  else
  return x*y*z,x+y+z #Return a different tuple
  #whitespace is optional
  end #End if statement
end #End function definition

Here, if z is not specified, then it's 0.

In [ ]:
x,y = test_function(1,2)

In [ ]:
x,y = test_function(1,2;z=3)

Notice that we also featured multiple return values.

In [ ]:
println(x); println(y)

The return type for multiple return values is a Tuple. The syntax for a tuple is `(x,y,z,...)` or inside of functions you can use the shorthand `x,y,z,...` as shown.

Note that functions in Julia are "first-class". This means that functions are just a type themselves. Therefore functions can make functions, you can store functions as variables, pass them as variables, etc. For example:

In [ ]:
function function_playtime(x) #z is an optional argument
    y = 2+x
    function test()
        2y # y is defined in the previous scope, so it's available here
    end
    z = test() * test()
    return z,test
end #End function definition
z,test = function_playtime(2)

In [ ]:
test()

In [ ]:
8

Notice that `test()` does not get passed in `y` but knows what `y` is. This is due to the function scoping rules: an inner function can know the variables defined in the same scope as the function. This rule is recursive, leading us to the conclusion that the top level scope is global. Yes, that means

In [ ]:
a = 2

defines a global variable. We will go into more detail on this.

Lastly we show the anonymous function syntax. This allows you to define a function inline.

In [ ]:
g = (x,y) -> 2x+y

Unlike named functions, `g` is simply a function in a variable and can be overwritten at any time:

In [ ]:
g = (x) -> 2x

An anonymous function cannot have more than 1 dispatch. However, as of v0.5, they are compiled and thus do not have any performance disadvantages from named functions. We will encounter anonymous functions later on when we will use numerical optimizers, for example.

## Type Declaration Syntax

A type is what in many other languages is an "object". If that is a foreign concept, thing of a type as a thing which has named components. A type is the idea for what the thing is, while an instantiation of the type is a specific one. For example, you can think of a car as having an make and a model. So that means a Toyota RAV4 is an instantiation of the car type.

In Julia, we would define the car `struct` as follows. It is very similar to a matlab struct or a `C++` struct or a `fortran` data type:

In [16]:
struct Car
    make
    model
end

We could then make the instance of a car as follows:

In [17]:
mycar = Car("Toyota","Rav4")

Car("Toyota", "Rav4")

Here I introduced the string syntax for Julia which uses "..." (like most other languages, I'm glaring at you MATLAB). I can grab the "fields" of my type using the `.` syntax:

In [18]:
mycar.make

"Toyota"

To "enhance Julia's performance", one usually likes to make the **typing stricter**. For example, we can define a WorkshopParticipant (notice the convention for types is capital letters, CamelCase) as having a name and a field. The name will be a string and the field will be a Symbol type, (defined by :Symbol, which we will go into plenty more detail later).

In [19]:
struct WorkshopParticipant
    name::String
    field::Symbol
end
tony = WorkshopParticipant("Tony",:physics)

WorkshopParticipant("Tony", :physics)

As with functions, types can be set "parametrically". For example, we can have an StaffMember have a name and a field, but also an age. We can allow this age to be any Number type as follows:

In [20]:
struct StaffMember{T<:Number}
    name::String
    field::Symbol
    age::T
end
ter = StaffMember("Terry",:football,17)

StaffMember{Int64}("Terry", :football, 17)

In [21]:
# notice that we cannot change terry:
ter.name = "Ptere"

LoadError: [91mtype StaffMember is immutable[39m

In [23]:
mutable struct StaffM{T<:Number}
    name::String
    field::Symbol
    age::T
end
ter = StaffM("Terry",:football,17)
ter.age+=1
ter

StaffM{Int64}("Terry", :football, 18)

The rules for parametric typing is the same as for functions. Note that most of Julia's types, like Float64 and Int, are natively defined in Julia in this manner. This means that **there's no limit for user defined types**, only your imagination. Indeed, many of Julia's features first start out as a prototyping package before it's ever moved into Base (the Julia library that ships as the Base module in every installation).

Lastly, there exist abstract types. These types cannot be instantiated but are used to build the type hierarchy. You've already seen one abstract type, Number. We can define one for Person using the Abstract keyword

In [28]:
abstract type Person end

Then we can set types as a subtype of person

In [29]:
type Student <: Person
    name
    grade
end

You can define type heirarchies on abstract types. See the beautiful explanation at: http://docs.julialang.org/en/latest/manual/types/#abstract-types

In [30]:
abstract type AbstractStudent <: Person end

Another "version" of type is `immutable`. When one uses `immutable`, the fields of the type cannot be changed. However, Julia will automatically stack allocate immutable types, whereas standard types are heap allocated. If this is unfamiliar terminology, then think of this as meaning that immutable types are able to be stored closer to the CPU and have less cost for memory access (this is a detail not present in many scripting languages). Many things like Julia's built-in Number types are defined as `immutable` in order to give good performance.

In [31]:
immutable Field
    name
    school
end
ds = Field(:DataScience,[:PhysicalScience;:ComputerScience])

Field(:DataScience, Symbol[:PhysicalScience, :ComputerScience])

----
#### Question 3

Can you explain this interesting quirk? This Field is immutable, meaning that `ds.name` and `ds.school` cannot be changed:

In [ ]:
ds.name = :ComputationalStatistics

However, the following is allowed:

In [ ]:
push!(ds.school,:BiologicalScience)
ds.school

(Hint: recall that an array is not the values itself, but a pointer to the memory of the values)

-----------------------

One important detail in Julia is that everything is a type (and every piece of code is an Expression type, more on this later). Thus functions are also types, which we can access the fields of. Not only is everything compiled down to C, but all of the "C parts" are always accessible. For example, we can, if we so choose, get a function pointer:

In [ ]:
foo(x) = 2x
first(methods(foo)).lambda_template.fptr

------------------------------

## Problems

Try the new few problems in the Basic Problems

## Some Basic Types

Julia provides many basic types. Indeed, you will come to know Julia as a system of multiple dispatch on types, meaning that the interaction of types with functions is core to the design.

### Lazy Iterator Types

While MATLAB or Python has easy functions for building arrays, Julia tends to side-step the actual "array" part with specially made types. One such example are ranges. To define a range, use the `start:stepsize:end` syntax. For example:

In [50]:
a = 1:5
println(a)
b = 1:2:10
println(b)

1:5
1:2:9


We can use them like any array. For example:

In [51]:
println(a[2]); println(b[3])

2
5


But what is `b`?

In [52]:
println(typeof(b))

StepRange{Int64,Int64}


`b` isn't an array, it's a StepRange. A StepRange has the ability to act like an array using its fields:

In [53]:
fieldnames(StepRange)

3-element Array{Symbol,1}:
 :start
 :step 
 :stop 

Note that at any time we can get the array from these kinds of type via the `collect` function:

In [54]:
c = collect(a)

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

The reason why lazy iterator types are preferred is that they do not do the computations until it's absolutely necessary, and they take up much less space. We can check this with `@time`:

In [55]:
@time a = 1:100000
@time a = 1:100
@time b = collect(1:100000);

  0.000006 seconds (85 allocations: 6.467 KiB)
  0.000001 seconds (5 allocations: 192 bytes)
  0.000292 seconds (8 allocations: 781.547 KiB)


Notice that the amount of time the range takes is much shorter. This is mostly because there is a lot less memory allocation needed: only a `StepRange` is built, and all that holds is the three numbers. However, `b` has to hold `100000` numbers, leading to the huge difference.

### Dictionaries 

Another common type is the Dictionary. It allows you to access (key,value) pairs in a named manner. For example:

In [ ]:
d = Dict(:test=>2,"silly"=>:suit)
println(d[:test])
println(d["silly"])

### Tuples

Tuples are immutable arrays. That means they can't be changed. However, they are super fast. They are made with the `(x,y,z,...)` syntax and are the standard return type of functions which return more than one object.

In [ ]:
tup = (2.,3) # Don't have to match types
x,y = (3.0,"hi") # Can separate a tuple to multiple variables

## Problems

Try problems 8-11 in the Basic Problems

## Metaprogramming

Metaprogramming is a huge feature of Julia. The key idea is that every statement in Julia is of the type `Expression`. Julia operates by building an Abstract Syntax Tree (AST) from the Expressions. You've already been exposed to this a little bit: a `Symbol` (like `:PhysicalSciences` is not a string because it is part of the AST, and thus is part of the parsing/expression structure. One interesting thing is that symbol comparisons are O(1) while string comparisons, like always, are O(n)) is part of this, and macros (the weird functions with an `@`) are functions on expressions.

* This is different from a `C++` preprocessor, which passes over your code and *replaces* some text with another piece of text, before continuing to compile it to machine code.
* The difference is that parts of the AST are like any other data type in julia. You can *programmatically* modify *your code*.
* Julia can *write code* in this sense.
* Important parts of the standard library do this, in order to avoid multiple loops for example. `sub2ind` is an example.

### Metaprogramming 101

* We will go over a couple of simple examples from the [manual on this](https://docs.julialang.org/en/latest/manual/metaprogramming/).
* We always start with a string, which we then *parse*, i.e. read.

In [57]:
prog = "1 + 1"
ex1 = parse(prog)
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1
  typ: Any


In [58]:
ex1.head

:call

In [59]:
ex1.args

3-element Array{Any,1}:
  :+
 1  
 1  

In [60]:
# directly constructing
ex2 = Expr(:call, :+, 1, 1)
@test ex1 == ex2

Test Passed


### Use of `:`

1. create a `Symbol`
    ```julia
    julia> :foo
    :foo

    julia> typeof(ans)
    Symbol
    ```
1. create an expression
    ```julia
    julia> ex = :(a+b*c+1)
    :(a + b * c + 1)

    julia> typeof(ex)
    Expr
    ```

### Interpolation

* we can *insert* values into expressions
* use the `$` operator as with string interpolation:
    ```julia
    julia> a = 1;

    julia> ex = :($a + b)
    :(1 + b)
    ```
    
### `eval` evaluates an expression

* the evaluation takes the values passed at *expression construction time*
* other symbols (:b) are looked up in the current scope:

In [62]:
a = 1;
ex = Expr(:call, :+, a, :b)
a = 0; b = 2;
eval(ex)

3

### Macros

* key feature of julia
* A macro maps a tuple of arguments to an `Expr`ession
* The resulting expression is directly compiled, without an `eval` call.
* the fundamental thing is that macros allow us to modify/generate code **before** the program actually runs.

In [63]:
# simplest macro?
macro sayhello(name)
    return :( println("Hello, ", $name) )  # note: returns an expression
end
@sayhello "you"

Hello, you


In [65]:
# look inside a macro
ex = @macroexpand @sayhello "Herbert"
println(ex)
typeof(ex)

(Main.println)("Hello, ", "Herbert")


Expr

search: end endof endswith ENDIAN_BOM send append! QuoteNode RangeIndex getindex



In [68]:
## two step illustration: parse time vs run time
# prelim: you can also give functions of Expr!
macro twostep(arg)
   println("I execute at parse time. The argument is: ", arg)
   return :(println("I execute at runtime. The argument is: ", $arg))
end
 ex = macroexpand( :(@twostep :(1, 2, 3)) );

I execute at parse time. The argument is: $(Expr(:quote, :((1, 2, 3))))


In [69]:
# runtime:
eval(ex)

I execute at runtime. The argument is: (1, 2, 3)


In [ ]:
# Thus you can think of metaprogramming as 
# "code which takes in code and outputs code". 
# One basic example is the `@time` macro: 

macro my_time(ex)
  return quote
    local t0 = time()
    local val = $ex
    local t1 = time()
    println("elapsed time: ", t1-t0, " seconds")
    val
  end
end

In [ ]:
@my_time (macro with 1 method)